In [24]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.gridspec as gridspec
import cantera as ct
from sklearn.linear_model import LinearRegression
import os
import concurrent.futures

def reverse_Arrhenius(a):   
    reaction_idx=a
    gas = ct.Solution(cti_file, 'gas')
    surf = ct.Interface(cti_file,'Rh331', [gas])

    t=np.linspace(100,1000, 50)
    k_forward=np.zeros(len(t))
    k_reverse=np.zeros(len(t))
    for i in range(len(t)):
      gas.TPX =t[i]+273, ct.one_atm, 'N2:1'
      surf.TP = t[i]+273, ct.one_atm
      surf.coverages = {'Rh(331)':1}
      k_reverse[i]=surf.reverse_rate_constants[reaction_idx]
      k_forward[i]=surf.forward_rate_constants[reaction_idx]
   
    T_reciprok=1/(t+273)    
    log_k_forward=np.log(k_forward)
    log_k_reverse=np.log(k_reverse)
    log_k_reverse=log_k_reverse.reshape(len(k_reverse),1)
    T_reciprok=T_reciprok.reshape(len(T_reciprok),1)

    #Linear regression of an Arrhenius expression to the calculated rate constant
    model =LinearRegression(fit_intercept=True)
    model.fit(T_reciprok,log_k_reverse)
    yfit=model.predict(log_k_reverse)
    Ea_R=-float(model.coef_[0])
    ln_k0=float(model.intercept_[0])
    activation_barrier=np.round(Ea_R*ct.gas_constant,3)
    pre_exponential=float(np.format_float_scientific(np.exp(ln_k0)*10,precision=3))
    #print(surf.reaction_equations()[a])
    #print(pre_exponential,activation_barrier)
    return pre_exponential, activation_barrier

cti_file='WGS_on_Rh_HRHT.yaml'
gas = ct.Solution(cti_file, 'gas')
gas.TPX = 373, ct.one_atm, 'N2:1'

# import the surface model
surf = ct.Interface(cti_file,'Rh331', [gas])
surf.TP = 273, ct.one_atm
surf.coverages = {'Rh(331)':1}

index=[5,7,13,15]
for i in index:
    rev_params=reverse_Arrhenius(i)
    print(rev_params[0])
    print(rev_params[1]/1e6)
    print(surf.reaction_equations()[i])

# idx=[]
# for i in range(surf.n_reactions):
#     print(i)
#     print(surf.reaction_equations()[i])


1.337e+22
149.162252349
CO2(331) <=> CO(331) + O(331)
1.44e+24
112.60096576000001
OH(331) + Rh(331) <=> H(331) + O(331)
1.45e+22
56.832994938
2 OH(331) <=> H2O(331) + O(331)
2.322e+29
199.82164048500002
CO(331) + Rh(331) <=> C(331) + O(331)


/var/folders/ct/t72gd9jn14l9bbfml_gschyh0000gn/T/ipykernel_47005/3924008731.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  Ea_R=-float(model.coef_[0])
/var/folders/ct/t72gd9jn14l9bbfml_gschyh0000gn/T/ipykernel_47005/3924008731.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  Ea_R=-float(model.coef_[0])
/var/folders/ct/t72gd9jn14l9bbfml_gschyh0000gn/T/ipykernel_47005/3924008731.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  Ea_R=-float(model.coef_[0]

In [19]:
cti_file='Multifacet_mechanism_flipped_reactions_covdep.yaml'
gas = ct.Solution(cti_file, 'gas')
gas.TPX = 373, ct.one_atm, 'Ar:73.9695'

# import the surface model
surf = ct.Interface(cti_file,'surface4', [gas])
surf.TP = 273, ct.one_atm
surf.coverages = {'site(110)':1}

for i in range(surf.n_reactions):
    print(surf.delta_standard_enthalpy[i]/1e6)

-477.54053895945714
-144.37562267087173
-164.15446041469198
51.16779084690005
88.1142076859575
